<a href="https://colab.research.google.com/github/06-preetinder/ai-waste-classification/blob/main/3_model_retraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile

zip_path = '/content/drive/MyDrive/ai-project/2_dataset.zip'
extract_path = '/content/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Same batch size
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_PATH = '/content/archive'  # Changing the path for second dataset

# Train and validation generators with strong augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

# Save class names
class_names = list(train_generator.class_indices.keys())

Found 2669 images belonging to 5 classes.
Found 666 images belonging to 5 classes.


In [10]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load model
model = load_model('/content/drive/MyDrive/ai-project/waste_classification_model.h5')
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
from sklearn.utils import class_weight
import numpy as np

# Get the class indices from the generator
class_indices = train_generator.class_indices
classes = list(class_indices.keys())

# Get class labels for each image
labels = train_generator.classes

# Compute weights
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)

# Convert to dictionary: {class_index: weight}
class_weights = dict(enumerate(weights))

print("Class Weights:", class_weights)

Class Weights: {0: np.float64(2.8243386243386244), 1: np.float64(0.8609677419354839), 2: np.float64(0.8609677419354839), 3: np.float64(0.8609677419354839), 4: np.float64(0.8609677419354839)}


In [12]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    patience=3,
    restore_best_weights=True
)

In [13]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop],
    class_weight=class_weights
)

Epoch 1/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step - accuracy: 0.5278 - loss: 1.2375

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


84/84 ━━━━━━━━━━━━━━━━━━━━ 54s 550ms/step - accuracy: 0.5282 - loss: 1.2356 - val_accuracy: 0.5886 - val_loss: 1.0701
Epoch 2/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 42s 502ms/step - accuracy: 0.6025 - loss: 0.8687 - val_accuracy: 0.5405 - val_loss: 1.0954
Epoch 3/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 81s 493ms/step - accuracy: 0.6760 - loss: 0.7562 - val_accuracy: 0.6336 - val_loss: 0.8879
Epoch 4/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 41s 493ms/step - accuracy: 0.6968 - loss: 0.7246 - val_accuracy: 0.6156 - val_loss: 0.9289
Epoch 5/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 41s 491ms/step - accuracy: 0.7010 - loss: 0.7458 - val_accuracy: 0.6171 - val_loss: 0.9796
Epoch 6/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 41s 491ms/step - accuracy: 0.7280 - loss: 0.6337 - val_accuracy: 0.5721 - val_loss: 1.0186


In [14]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    patience=5,
    restore_best_weights=True
)

In [15]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop],
    class_weight=class_weights
)

Epoch 1/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 42s 506ms/step - accuracy: 0.6906 - loss: 0.7329 - val_accuracy: 0.6036 - val_loss: 0.9615
Epoch 2/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 41s 495ms/step - accuracy: 0.6922 - loss: 0.6774 - val_accuracy: 0.6336 - val_loss: 0.8990
Epoch 3/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 45s 532ms/step - accuracy: 0.6971 - loss: 0.7302 - val_accuracy: 0.5901 - val_loss: 0.9997
Epoch 4/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 42s 495ms/step - accuracy: 0.6954 - loss: 0.7010 - val_accuracy: 0.5856 - val_loss: 1.0734
Epoch 5/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 82s 495ms/step - accuracy: 0.7321 - loss: 0.6199 - val_accuracy: 0.6291 - val_loss: 0.9128
Epoch 6/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 45s 532ms/step - accuracy: 0.7291 - loss: 0.6269 - val_accuracy: 0.6366 - val_loss: 0.8548
Epoch 7/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 42s 496ms/step - accuracy: 0.7614 - loss: 0.5559 - val_accuracy: 0.6547 - val_loss: 0.8506
Epoch 8/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 41s 494ms/step - accuracy: 0.7645 - loss: 0.5346 - val_accu

In [18]:
!pip install split-folders

In [19]:
import splitfolders

input_folder = '/content/archive'
output_folder = '/content/split'

splitfolders.ratio(
    input_folder,
    output=output_folder,
    seed=42,
    ratio=(0.8, 0.1, 0.1),
    move=True
)

Copying files: 3335 files [00:00, 23643.92 files/s]


In [20]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/split/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Evaluate model
test_loss, test_acc = model.evaluate(test_generator)
print(f"✅ Test Accuracy: {test_acc:.4f}")
print(f"❌ Test Loss: {test_loss:.4f}")

Found 337 images belonging to 5 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 165ms/step - accuracy: 0.8298 - loss: 0.4540
✅ Test Accuracy: 0.8309
❌ Test Loss: 0.4729


In [21]:
model.save("waste_classification_2.0.h5")